###Packages & data

In [0]:
# Notebook packages
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt

from sklearn import datasets, linear_model
from sklearn import metrics as met
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
import sklearn.model_selection as ms
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV

In [0]:
from google.colab import drive
drive.mount('/content/drive')

# Data load
features = pd.read_csv('/content/drive/My Drive/Sendy_data/Train.csv')
rider_info = pd.read_csv('/content/drive/My Drive/Sendy_data/Riders.csv')
v_features = pd.read_csv('/content/drive/My Drive/Sendy_data/Test.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


###Initial exploration

In [0]:
print(v_features.shape)
print(' ')
v_features.dtypes

(7068, 25)
 


Order No                                 object
User Id                                  object
Vehicle Type                             object
Platform Type                             int64
Personal or Business                     object
Placement - Day of Month                  int64
Placement - Weekday (Mo = 1)              int64
Placement - Time                         object
Confirmation - Day of Month               int64
Confirmation - Weekday (Mo = 1)           int64
Confirmation - Time                      object
Arrival at Pickup - Day of Month          int64
Arrival at Pickup - Weekday (Mo = 1)      int64
Arrival at Pickup - Time                 object
Pickup - Day of Month                     int64
Pickup - Weekday (Mo = 1)                 int64
Pickup - Time                            object
Distance (KM)                             int64
Temperature                             float64
Precipitation in millimeters            float64
Pickup Lat                              

In [0]:
print(features.shape)
print(' ')
features.dtypes

(21201, 29)
 


Order No                                      object
User Id                                       object
Vehicle Type                                  object
Platform Type                                  int64
Personal or Business                          object
Placement - Day of Month                       int64
Placement - Weekday (Mo = 1)                   int64
Placement - Time                              object
Confirmation - Day of Month                    int64
Confirmation - Weekday (Mo = 1)                int64
Confirmation - Time                           object
Arrival at Pickup - Day of Month               int64
Arrival at Pickup - Weekday (Mo = 1)           int64
Arrival at Pickup - Time                      object
Pickup - Day of Month                          int64
Pickup - Weekday (Mo = 1)                      int64
Pickup - Time                                 object
Arrival at Destination - Day of Month          int64
Arrival at Destination - Weekday (Mo = 1)     

In [0]:
print('How many null values were in each variable?')
print('The number of null values were as follows:')
Nulls = v_features.isnull().sum()
print(Nulls)

How many null values were in each variable?
The number of null values were as follows:
Order No                                   0
User Id                                    0
Vehicle Type                               0
Platform Type                              0
Personal or Business                       0
Placement - Day of Month                   0
Placement - Weekday (Mo = 1)               0
Placement - Time                           0
Confirmation - Day of Month                0
Confirmation - Weekday (Mo = 1)            0
Confirmation - Time                        0
Arrival at Pickup - Day of Month           0
Arrival at Pickup - Weekday (Mo = 1)       0
Arrival at Pickup - Time                   0
Pickup - Day of Month                      0
Pickup - Weekday (Mo = 1)                  0
Pickup - Time                              0
Distance (KM)                              0
Temperature                             1437
Precipitation in millimeters            6869
Pickup Lat   

In [0]:
print('How many null values were in each variable?')
print('The number of null values were as follows:')
Nulls = features.isnull().sum()
print(Nulls)

How many null values were in each variable?
The number of null values were as follows:
Order No                                         0
User Id                                          0
Vehicle Type                                     0
Platform Type                                    0
Personal or Business                             0
Placement - Day of Month                         0
Placement - Weekday (Mo = 1)                     0
Placement - Time                                 0
Confirmation - Day of Month                      0
Confirmation - Weekday (Mo = 1)                  0
Confirmation - Time                              0
Arrival at Pickup - Day of Month                 0
Arrival at Pickup - Weekday (Mo = 1)             0
Arrival at Pickup - Time                         0
Pickup - Day of Month                            0
Pickup - Weekday (Mo = 1)                        0
Pickup - Time                                    0
Arrival at Destination - Day of Month         

In [0]:
rider_info.dtypes

Rider Id           object
No_Of_Orders        int64
Age                 int64
Average_Rating    float64
No_of_Ratings       int64
dtype: object

In [0]:
print('How many null values were in each variable?')
print('The number of null values were as follows:')
Nulls = rider_info.isnull().sum()
print(Nulls)
print(' ')

How many null values were in each variable?
The number of null values were as follows:
Rider Id          0
No_Of_Orders      0
Age               0
Average_Rating    0
No_of_Ratings     0
dtype: int64
 


###Initial Preprocessing

####Features

In [0]:
print(features.shape)
print(' ')
features.head(10)

(21201, 29)
 


,Order No,User Id,Vehicle Type,Platform Type,Personal or Business,Placement - Day of Month,Placement - Weekday (Mo = 1),Placement - Time,Confirmation - Day of Month,Confirmation - Weekday (Mo = 1),Confirmation - Time,Arrival at Pickup - Day of Month,Arrival at Pickup - Weekday (Mo = 1),Arrival at Pickup - Time,Pickup - Day of Month,Pickup - Weekday (Mo = 1),Pickup - Time,Arrival at Destination - Day of Month,Arrival at Destination - Weekday (Mo = 1),Arrival at Destination - Time,Distance (KM),Temperature,Precipitation in millimeters,Pickup Lat,Pickup Long,Destination Lat,Destination Long,Rider Id,Time from Pickup to Arrival
0,Order_No_4211,User_Id_633,Bike,3,Business,9,5,9:35:46 AM,9,5,9:40:10 AM,9,5,10:04:47 AM,9,5,10:27:30 AM,9,5,10:39:55 AM,4,20.4,NaN,-1.317755,36.830370,-1.300406,36.829741,Rider_Id_432,745
1,Order_No_25375,User_Id_2285,Bike,3,Personal,12,5,11:16:16 AM,12,5,11:23:21 AM,12,5,11:40:22 AM,12,5,11:44:09 AM,12,5,12:17:22 PM,16,26.4,NaN,-1.351453,36.899315,-1.295004,36.814358,Rider_Id_856,1993
2,Order_No_1899,User_Id_265,Bike,3,Business,30,2,12:39:25 PM,30,2,12:42:44 PM,30,2,12:49:34 PM,30,2,12:53:03 PM,30,2,1:00:38 PM,3,NaN,NaN,-1.308284,36.843419,-1.300921,36.828195,Rider_Id_155,455
3,Order_No_9336,User_Id_1402,Bike,3,Business,15,5,9:25:34 AM,15,5,9:26:05 AM,15,5,9:37:56 AM,15,5,9:43:06 AM,15,5,10:05:27 AM,9,19.2,NaN,-1.281301,36.832396,-1.257147,36.795063,Rider_Id_855,1341
4,Order_No_27883,User_Id_1737,Bike,1,Personal,13,1,9:55:18 AM,13,1,9:56:18 AM,13,1,10:03:53 AM,13,1,10:05:23 AM,13,1,10:25:37 AM,9,15.4,NaN,-1.266597,36.792118,-1.295041,36.809817,Rider_Id_770,1214
5,Order_No_7408,User_Id_1342,Bike,3,Business,14,5,3:07:35 PM,14,5,3:08:57 PM,14,5,3:21:36 PM,14,5,3:30:30 PM,14,5,4:23:41 PM,9,27.2,NaN,-1.302583,36.767081,-1.257309,36.806008,Rider_Id_124,3191
6,Order_No_22680,User_Id_2803,Bike,3,Business,9,5,9:33:45 AM,9,5,9:49:47 AM,9,5,9:53:12 AM,9,5,9:56:45 AM,9,5,10:19:45 AM,5,20.3,NaN,-1.279395,36.825364,-1.276574,36.851365,Rider_Id_114,1380
7,Order_No_21578,User_Id_1075,Bike,3,Business,11,1,2:13:01 PM,11,1,2:14:13 PM,11,1,2:21:33 PM,11,1,2:22:40 PM,11,1,2:33:26 PM,3,28.7,NaN,-1.252796,36.800313,-1.255189,36.782203,Rider_Id_913,646
8,Order_No_5234,User_Id_733,Bike,3,Business,30,2,11:10:44 AM,30,2,11:15:49 AM,30,2,12:13:18 PM,30,2,12:22:57 PM,30,2,1:19:35 PM,9,NaN,NaN,-1.255189,36.782203,-1.300255,36.825657,Rider_Id_394,3398
9,Order_No_1768,User_Id_2112,Bike,3,Business,23,5,4:48:54 PM,23,5,5:17:56 PM,23,5,5:32:41 PM,23,5,5:34:38 PM,23,5,6:31:57 PM,14,24.6,NaN,-1.225322,36.808550,-1.215601,36.891686,Rider_Id_660,3439


In [0]:
# First drop irrelevant features and features with too many missing values (i.e. Precipitation in millimeters):
features = features.drop(['User Id'], axis = 1)
features = features.drop(['Vehicle Type'], axis = 1)
features = features.drop(['Precipitation in millimeters'], axis = 1)
features = features.drop(['Arrival at Destination - Day of Month'], axis = 1)
features = features.drop(['Arrival at Destination - Weekday (Mo = 1)'], axis = 1)
features = features.drop(['Arrival at Destination - Time'], axis = 1)

# then convert values to datetime
features['Placement - Time'] = pd.to_datetime(features['Placement - Time'])
features['Confirmation - Time'] = pd.to_datetime(features['Confirmation - Time'])
features['Arrival at Pickup - Time'] = pd.to_datetime(features['Arrival at Pickup - Time'])
features['Pickup - Time'] = pd.to_datetime(features['Pickup - Time'])

print(features.shape)

(21201, 23)


In [0]:
# Dummify personal and business
dummies = pd.get_dummies(features['Personal or Business'], prefix='CustomerType')
dummies = pd.DataFrame(dummies, index=features.index)

print(dummies.shape)
dummies.head(10)

(21201, 2)


,CustomerType_Business,CustomerType_Personal
0,1,0
1,0,1
2,1,0
3,1,0
4,0,1
5,1,0
6,1,0
7,1,0
8,1,0
9,1,0


In [0]:
# Join rider_info and dummy features with the dataset
features = features.drop(['Personal or Business'], axis = 1)

features = features.join(dummies, how='left', rsuffix='_other')
features = features.join(rider_info, how='left', rsuffix='_other')

features = features.drop(['No_of_Ratings'], axis = 1)
features = features.drop(['No_Of_Orders'], axis = 1)
features = features.drop(['Rider Id'], axis = 1)
features = features.drop(['Rider Id_other'], axis = 1)

print(features.shape)

(21201, 25)


In [0]:
# Extract turnaround times from the dataset (A key assumption is that confirmation, pickup and delivery all happened on the same day)
Time_taken_to_confirm = features['Confirmation - Time'] - features['Placement - Time']
Time_taken_to_pickup = features['Pickup - Time'] - features['Arrival at Pickup - Time']

Time_taken_to_confirm = Time_taken_to_confirm.dt.total_seconds()
Time_taken_to_pickup = Time_taken_to_pickup.dt.total_seconds()

features.loc[:,'Time_taken_to_confirm'] = Time_taken_to_confirm
features.loc[:,'Time_taken_to_pickup'] = Time_taken_to_pickup

features = features.drop(['Placement - Time'], axis = 1)
features = features.drop(['Confirmation - Time'], axis = 1)
features = features.drop(['Arrival at Pickup - Time'], axis = 1)
features = features.drop(['Pickup - Time'], axis = 1)

print(features.shape)

(21201, 23)


In [0]:
# Set the index
features = features.set_index('Order No')
features.head(10)

,Platform Type,Placement - Day of Month,Placement - Weekday (Mo = 1),Confirmation - Day of Month,Confirmation - Weekday (Mo = 1),Arrival at Pickup - Day of Month,Arrival at Pickup - Weekday (Mo = 1),Pickup - Day of Month,Pickup - Weekday (Mo = 1),Distance (KM),Temperature,Pickup Lat,Pickup Long,Destination Lat,Destination Long,Time from Pickup to Arrival,CustomerType_Business,CustomerType_Personal,Age,Average_Rating,Time_taken_to_confirm,Time_taken_to_pickup
Order No,,,,,,,,,,,,,,,,,,,,,,
Order_No_4211,3,9,5,9,5,9,5,9,5,4,20.4,-1.317755,36.830370,-1.300406,36.829741,745,1,0,2298.0,14.0,264.0,1363.0
Order_No_25375,3,12,5,12,5,12,5,12,5,16,26.4,-1.351453,36.899315,-1.295004,36.814358,1993,0,1,951.0,13.5,425.0,227.0
Order_No_1899,3,30,2,30,2,30,2,30,2,3,NaN,-1.308284,36.843419,-1.300921,36.828195,455,1,0,821.0,14.3,199.0,209.0
Order_No_9336,3,15,5,15,5,15,5,15,5,9,19.2,-1.281301,36.832396,-1.257147,36.795063,1341,1,0,980.0,12.5,31.0,310.0
Order_No_27883,1,13,1,13,1,13,1,13,1,9,15.4,-1.266597,36.792118,-1.295041,36.809817,1214,0,1,1113.0,13.7,60.0,90.0
Order_No_7408,3,14,5,14,5,14,5,14,5,9,27.2,-1.302583,36.767081,-1.257309,36.806008,3191,1,0,1798.0,13.2,82.0,534.0
Order_No_22680,3,9,5,9,5,9,5,9,5,5,20.3,-1.279395,36.825364,-1.276574,36.851365,1380,1,0,1304.0,13.4,962.0,213.0
Order_No_21578,3,11,1,11,1,11,1,11,1,3,28.7,-1.252796,36.800313,-1.255189,36.782203,646,1,0,2124.0,14.1,72.0,67.0
Order_No_5234,3,30,2,30,2,30,2,30,2,9,NaN,-1.255189,36.782203,-1.300255,36.825657,3398,1,0,1436.0,14.2,305.0,579.0


####v_features

In [0]:
# Repeat steps

In [0]:
v_features = v_features.drop(['User Id'], axis = 1)
v_features = v_features.drop(['Vehicle Type'], axis = 1)
v_features = v_features.drop(['Precipitation in millimeters'], axis = 1)

v_features['Placement - Time'] = pd.to_datetime(v_features['Placement - Time'])
v_features['Confirmation - Time'] = pd.to_datetime(v_features['Confirmation - Time'])
v_features['Arrival at Pickup - Time'] = pd.to_datetime(v_features['Arrival at Pickup - Time'])
v_features['Pickup - Time'] = pd.to_datetime(v_features['Pickup - Time'])

print(v_features.shape)

(7068, 22)


In [0]:
v_dummies = pd.get_dummies(v_features['Personal or Business'], prefix='CustomerType')
v_dummies = pd.DataFrame(v_dummies, index=v_features.index)

print(v_dummies.shape)

(7068, 2)


In [0]:
v_features = v_features.drop(['Personal or Business'], axis = 1)

v_features = v_features.join(dummies, how='left', rsuffix='_other')
v_features = v_features.join(rider_info, how='left', rsuffix='_other')

v_features = v_features.drop(['No_of_Ratings'], axis = 1)
v_features = v_features.drop(['No_Of_Orders'], axis = 1)
v_features = v_features.drop(['Rider Id'], axis = 1)
v_features = v_features.drop(['Rider Id_other'], axis = 1)
print(v_features.shape)

(7068, 24)


In [0]:
Time_taken_to_confirm_v = v_features['Confirmation - Time'] - v_features['Placement - Time']
Time_taken_to_pickup_v = v_features['Pickup - Time'] - v_features['Arrival at Pickup - Time']

Time_taken_to_confirm_v = Time_taken_to_confirm_v.dt.total_seconds()
Time_taken_to_pickup_v = Time_taken_to_pickup_v.dt.total_seconds()

v_features.loc[:,'Time_taken_to_confirm'] = Time_taken_to_confirm_v
v_features.loc[:,'Time_taken_to_pickup'] = Time_taken_to_pickup_v

v_features = v_features.drop(['Placement - Time'], axis = 1)
v_features = v_features.drop(['Confirmation - Time'], axis = 1)
v_features = v_features.drop(['Arrival at Pickup - Time'], axis = 1)
v_features = v_features.drop(['Pickup - Time'], axis = 1)

print(v_features.shape)

(7068, 22)


In [0]:
v_features = v_features.set_index('Order No')
v_features.head(10)

,Platform Type,Placement - Day of Month,Placement - Weekday (Mo = 1),Confirmation - Day of Month,Confirmation - Weekday (Mo = 1),Arrival at Pickup - Day of Month,Arrival at Pickup - Weekday (Mo = 1),Pickup - Day of Month,Pickup - Weekday (Mo = 1),Distance (KM),Temperature,Pickup Lat,Pickup Long,Destination Lat,Destination Long,CustomerType_Business,CustomerType_Personal,Age,Average_Rating,Time_taken_to_confirm,Time_taken_to_pickup
Order No,,,,,,,,,,,,,,,,,,,,,
Order_No_19248,3,27,3,27,3,27,3,27,3,8,NaN,-1.333275,36.870815,-1.305249,36.822390,1,0,2298.0,14.0,19.0,823.0
Order_No_12736,3,17,5,17,5,17,5,17,5,5,NaN,-1.272639,36.794723,-1.277007,36.823907,0,1,951.0,13.5,102.0,310.0
Order_No_768,3,27,4,27,4,27,4,27,4,5,22.8,-1.290894,36.822971,-1.276574,36.851365,1,0,821.0,14.3,1011.0,1474.0
Order_No_15332,3,17,1,17,1,17,1,17,1,5,24.5,-1.290503,36.809646,-1.303382,36.790658,1,0,980.0,12.5,112.0,851.0
Order_No_21373,3,11,2,11,2,11,2,11,2,6,24.4,-1.281081,36.814423,-1.266467,36.792161,0,1,1113.0,13.7,257.0,525.0
Order_No_14573,1,13,1,13,1,13,1,13,1,16,19.3,-1.256606,36.795974,-1.223983,36.898452,1,0,1798.0,13.2,4.0,136.0
Order_No_6731,2,17,3,17,3,17,3,17,3,18,20.9,-1.225272,36.875672,-1.304713,36.808955,1,0,1304.0,13.4,21.0,689.0
Order_No_18436,3,28,4,28,4,28,4,28,4,8,22.7,-1.273539,36.833775,-1.297299,36.789446,1,0,2124.0,14.1,93.0,68.0
Order_No_2288,3,28,4,28,4,28,4,28,4,8,19.4,-1.255189,36.782203,-1.285770,36.759172,1,0,1436.0,14.2,19.0,198.0


####Label

In [0]:
label = features['Time from Pickup to Arrival']
features = features.drop(['Time from Pickup to Arrival'], axis = 1)


###Filling NaN values

In [0]:
# For features
features['Temperature'] = features['Temperature'].fillna(features['Temperature'].median())
# To handle the remaining missing values(features['Precipitation in millimeters']):
features = features.fillna(0)

In [0]:
# For v_features
v_features['Temperature'] = v_features['Temperature'].fillna(v_features['Temperature'].median())
v_features = v_features.fillna(0)

###Modelling with CatBoost

In [0]:
!pip install catboost
from catboost import CatBoostRegressor,Pool

train_pool = Pool(features, 
                  label, 
                  cat_features=['Platform Type',
                                'Placement - Day of Month',
                                'Placement - Weekday (Mo = 1)',
                                'Confirmation - Day of Month',
                                'Confirmation - Weekday (Mo = 1)',
                                'Arrival at Pickup - Day of Month',
                                'Arrival at Pickup - Weekday (Mo = 1)',
                                'Pickup - Day of Month',
                                'Pickup - Weekday (Mo = 1)'])

validate_pool = Pool(v_features, 
                 cat_features=['Platform Type',
                                'Placement - Day of Month',
                                'Placement - Weekday (Mo = 1)',
                                'Confirmation - Day of Month',
                                'Confirmation - Weekday (Mo = 1)',
                                'Arrival at Pickup - Day of Month',
                                'Arrival at Pickup - Weekday (Mo = 1)',
                                'Pickup - Day of Month',
                                'Pickup - Weekday (Mo = 1)'])

In [0]:
model = CatBoostRegressor(iterations=150,
                          loss_function='RMSE')

model.fit(train_pool)

0:	learn: 1799.9754587	total: 65.9ms	remaining: 9.81s
1:	learn: 1758.1453396	total: 75.7ms	remaining: 5.6s
2:	learn: 1718.2547020	total: 86.1ms	remaining: 4.22s
3:	learn: 1678.5778349	total: 96.2ms	remaining: 3.51s
4:	learn: 1640.7714527	total: 105ms	remaining: 3.05s
5:	learn: 1604.3095736	total: 114ms	remaining: 2.75s
6:	learn: 1568.9534070	total: 124ms	remaining: 2.52s
7:	learn: 1535.0419489	total: 132ms	remaining: 2.34s
8:	learn: 1502.2634459	total: 140ms	remaining: 2.2s
9:	learn: 1470.7319975	total: 151ms	remaining: 2.11s
10:	learn: 1440.8964243	total: 160ms	remaining: 2.02s
11:	learn: 1411.9704695	total: 172ms	remaining: 1.97s
12:	learn: 1383.8170895	total: 182ms	remaining: 1.92s
13:	learn: 1357.0133295	total: 191ms	remaining: 1.85s
14:	learn: 1331.5310075	total: 201ms	remaining: 1.81s
15:	learn: 1306.9809387	total: 209ms	remaining: 1.75s
16:	learn: 1282.7802433	total: 219ms	remaining: 1.71s
17:	learn: 1259.8182531	total: 229ms	remaining: 1.68s
18:	learn: 1238.0468562	total: 237ms

In [0]:
answer = model.predict(validate_pool)

In [0]:
answer = pd.DataFrame(answer)
print(answer.shape)
# Save and download the prediction
from IPython.display import HTML
import base64  
import pandas as pd  

def create_download_link( df, title = "Download CSV file", filename = "data.csv"):  
    csv = df.to_csv(index =True)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

create_download_link(answer)

(7068, 1)
